#### \_\_annotations\_\_ ?

In [118]:
class User:
    id: int
    name: str
    something: "some random note"

In [119]:
User.__annotations__

{'id': int, 'name': str, 'something': 'some random note'}

#### Let's give it an \_\_init\_\_ method.

In [120]:
class BaseModel:
    def __init__(self, **kwargs):
        for key, value in kwargs.items():
            setattr(self, key, value)

class User(BaseModel):
    id: int
    name: str
    is_active:bool
        
user = User(id=123, name="john", is_active="yes")

In [121]:
user.id

123

In [122]:
user.name

'john'

In [123]:
user.is_active

'yes'

#### There's no type validations!

-

-

#### Introducing Metaclass
* type(class_name, base_classes, attributes)

In [124]:
class User:
    id=1
    name="john"
    is_active = True
User.__dict__

mappingproxy({'__module__': '__main__',
              'id': 1,
              'name': 'john',
              'is_active': True,
              '__dict__': <attribute '__dict__' of 'User' objects>,
              '__weakref__': <attribute '__weakref__' of 'User' objects>,
              '__doc__': None})

In [125]:
User2 = type("User", (), {'id':1, 'name':'john', 'is_active':True})
User2.__dict__

mappingproxy({'id': 1,
              'name': 'john',
              'is_active': True,
              '__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'User' objects>,
              '__weakref__': <attribute '__weakref__' of 'User' objects>,
              '__doc__': None})

#### Custom Metaclass

In [126]:
class MyMetaClass(type):
    def __new__(metacls, class_name, base_classes, attributes):
        print(f"{class_name} has attributes {attributes}")
        print(attributes)
        return super().__new__(metacls, class_name, base_classes, attributes)

In [127]:
class User3(metaclass=MyMetaClass):
    id=2
    name="john"
    is_active=True

User3 has attributes {'__module__': '__main__', '__qualname__': 'User3', 'id': 2, 'name': 'john', 'is_active': True}
{'__module__': '__main__', '__qualname__': 'User3', 'id': 2, 'name': 'john', 'is_active': True}


### Do you remember descriptors?
### We want to validate data as it is 'set'

In [128]:
class Descriptor:
    registry = {}
    def __init__(self, name, data_type):
        self.name = name
        self.data = data_type()
        self.data_type = data_type
    def __get__(self, instance, cls):
        return self.data
    def __set__(self, instance, value):
        ## we want to validate data as it is set
        self.data = value

class MyMetaClass(type):
    def __new__(metacls, class_name, base_classes, attributes):
        annotations = attributes.get('__annotations__',{})
        for name, data_type in annotations.items():
            attributes[name] = Descriptor(name=name, data_type=data_type)
        return super().__new__(metacls, class_name, base_classes, attributes)
    
class User3(metaclass=MyMetaClass):
    id:int
    name:str
    is_active:bool

In [129]:
User3.__dict__

mappingproxy({'__module__': '__main__',
              '__annotations__': {'id': int, 'name': str, 'is_active': bool},
              'id': <__main__.Descriptor at 0x108c531d0>,
              'name': <__main__.Descriptor at 0x108c53278>,
              'is_active': <__main__.Descriptor at 0x108c53208>,
              '__dict__': <attribute '__dict__' of 'User3' objects>,
              '__weakref__': <attribute '__weakref__' of 'User3' objects>,
              '__doc__': None})

In [130]:
user = User3()
user.name = "john"
user.id = 3
user.is_active = True

In [131]:
user.name

'john'

In [132]:
user.id

3

In [133]:
user.is_active

True

### Let's validate!

In [134]:
class Descriptor:
    registry = {}
    def __init__(self, name, data_type):
        self.name = name
        self.data = data_type()
        self.data_type = data_type
    def __get__(self, instance, cls):
        return self.data
    def __set__(self, instance, value):
        if isinstance(value, self.data_type):
            self.data = value
        else:
            raise Exception(f"{instance}.{self.name} is of type {self.data_type}, {value} of {type(value)} is given")

class MyMetaClass(type):
    def __new__(metacls, class_name, base_classes, attributes):
        annotations = attributes.get('__annotations__',{})
        for name, data_type in annotations.items():
            attributes[name] = Descriptor(name=name, data_type=data_type)
        return super().__new__(metacls, class_name, base_classes, attributes)
    
class User4(metaclass=MyMetaClass):
    id:int
    name:str
    is_active:bool      


In [135]:
user = User4()
user.name = 123

Exception: <__main__.User4 object at 0x108c53940>.name is of type <class 'str'>, 123 of <class 'int'> is given

### Final touch: the init

In [136]:
class Descriptor:
    registry = {}
    def __init__(self, name, data_type):
        self.name = name
        self.data = data_type()
        self.data_type = data_type
    def __get__(self, instance, cls):
        return self.data
    def __set__(self, instance, value):
        if isinstance(value, self.data_type):
            self.data = value
        else:
            raise Exception(f"{instance}.{self.name} is of type {self.data_type}, {value} of {type(value)} is given")

class MyMetaClass(type):
    def __new__(metacls, class_name, base_classes, attributes):
        annotations = attributes.get('__annotations__',{})
        for name, data_type in annotations.items():
            attributes[name] = Descriptor(name=name, data_type=data_type)
        return super().__new__(metacls, class_name, base_classes, attributes)
class Base(metaclass=MyMetaClass):
    def __init__(self, **kwargs):
        for key, value in kwargs.items():
            setattr(self, key, value)
class User5(Base):
    id:int
    name:str
    is_active:bool 

In [137]:
user = User5(id=1,name="john", is_active=True)

In [138]:
print(user.id)
print(user.name)
print(user.is_active)

1
john
True


In [139]:
user = User5(id=1,name=123, is_active=True)

Exception: <__main__.User5 object at 0x108c53978>.name is of type <class 'str'>, 123 of <class 'int'> is given

### Your turn! Could you make the code work?
```python
class InternalUser(Base):
    first_name = Column(str)
    last_name = Column(str)
    is_active = Column(bool)

user = InternalUser(first_name="john", last_name="peter", is_active = True)
print(user.first_name)
# john
print(user.last_name)
# peter
print(user.is_active)
# True
user.is_active = False
print(user.is_active)
# False
```

In [ ]:
class Column:
    pass


#  
#  
#  
#  
#  
#  
#  
#  
#  
#  
#  
#  
#  
#  
#  
#  
#  
#  
#  
#  
#  
#  
#  
#  
#  
#  
#  
#  


# Solution

In [140]:
class Column:
    def __init__(self, data_type):
        self.name = None
        self.data = data_type()
        self.data_type = data_type
    def __get__(self, instance, cls):
        return self.data
    def __set__(self, instance, value):
        if isinstance(value, self.data_type):
            self.data = value
        else:
            raise Exception(f"{instance}.{self.name} is of type {self.data_type}, {value} of {type(value)} is given")
class MyMetaClass(type):
    def __new__(metacls, class_name, base_classes, attributes):
        for attribute_name, attribute in attributes.items():
            if isinstance(attribute, Column):
                attribute.name = attribute_name
        return super().__new__(metacls, class_name, base_classes, attributes)
    
class Base(metaclass=MyMetaClass):
    def __init__(self, **kwargs):
        for key, value in kwargs.items():
            setattr(self, key, value)

class InternalUser(Base):
    first_name = Column(str)
    last_name = Column(str)
    is_active = Column(bool)

In [141]:
user = InternalUser(first_name="john", last_name="peter", is_active = True)
print(user.first_name)
# john
print(user.last_name)
# peter
print(user.is_active)
# True
user.is_active = False
print(user.is_active)

john
peter
True
False
